Iteration 1/500, Best Distance: 14.142135623730951
Iteration 51/500, Best Distance: 16.970562748477143
Iteration 101/500, Best Distance: 16.970562748477143
Iteration 151/500, Best Distance: 16.970562748477143
Iteration 201/500, Best Distance: 16.970562748477143
Iteration 251/500, Best Distance: 16.970562748477143
Iteration 301/500, Best Distance: 16.970562748477143
Iteration 351/500, Best Distance: 16.970562748477143
Iteration 401/500, Best Distance: 16.970562748477143
Iteration 451/500, Best Distance: 16.970562748477143
Best Route: [4, 0, 3, 2, 1], Best Distance: 16.970562748477143
